In [3]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from tqdm import tqdm

In [2]:
# --- CONFIG --- #
train_files = [
    "parsed_emg/subject_1/trial_11(right).csv",
    "parsed_emg/subject_1/trial_3(right).csv",
    "parsed_emg/subject_1/trial_5(right).csv",
    "parsed_emg/subject_1/trial_7(right).csv",
    "parsed_emg/subject_1/trial_9(right).csv",
]

test_file = "parsed_emg/subject_1/trial_1(right).csv"
label_col = 5

# --- Data Loader --- #
def load_csv(path):
    df = pd.read_csv(path, skiprows=1)
    df = df.drop(columns=df.columns[0], axis=1)
    X = df.iloc[:, 1].values.astype("float64").reshape(-1, 1)  # Make it 2D
    y = df.iloc[:, label_col].values.astype("int32")
    return X, y

In [6]:
X_train_list, y_train_list = [], []
for path in tqdm(train_files, desc="Loading training data"):
    X, y = load_csv(path)
    X_train_list.append(X)
    y_train_list.append(y)

X_train = np.vstack(X_train_list)
y_train = np.hstack(y_train_list)

# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# --- Train SVM --- #
print("Training SVM...")
model = LinearSVC(max_iter=1000)  # You can tune the kernel
model.fit(X_train, y_train)
print("Training complete.")

# --- Test --- #
X_test, y_test = load_csv(test_file)
y_pred = model.predict(X_test)

# --- Results --- #
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Loading training data: 100%|██████████| 5/5 [00:03<00:00,  1.52it/s]


Training SVM...
Training complete.

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     61829
           1       0.26      1.00      0.41     30726
           2       0.00      0.00      0.00     27894

    accuracy                           0.26    120449
   macro avg       0.09      0.33      0.14    120449
weighted avg       0.07      0.26      0.10    120449



c:\Users\Admin\VS Code (Python)\iMADE Stealth\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Admin\VS Code (Python)\iMADE Stealth\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Admin\VS Code (Python)\iMADE Stealth\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f